In [ ]:
%load_ext autoreload
%autoreload 2
import sys
import torch
from torch import nn
import numpy as np
from pathlib import Path
from typing import Dict

import matplotlib.pyplot as plt
gpu_id = 0

## Load matrix

In [ ]:
dir_matrix = torch.normal(0,1,size=(10,1000))
dir_matrix.requires_grad_(True)

In [ ]:
dir_matrix.shape, dir_matrix.device, dir_matrix.requires_grad, torch.linalg.norm(dir_matrix, dim=1, keepdim=True)

## erank function

In [ ]:
def erank_svd(matrix_A: torch.Tensor, center_matrix_A: bool=False) -> torch.Tensor:
        """Calculates the effective rank of a matrix.

        Args:
            matrix_A (torch.Tensor): Matrix of shape m x n. 
            center_matrix_A (bool): Center the matrix 

        Returns:
            torch.Tensor: Effective rank of matrix_A
        """
        assert matrix_A.ndim == 2
        _, s, _ = torch.linalg.svd(matrix_A, full_matrices=False)
        # normalizes input s -> scale independent!
        return torch.exp(torch.distributions.Categorical(s).entropy())

def erank_pca_lr(matrix_A: torch.Tensor, center_matrix_A: bool=False) -> torch.Tensor:
        """Calculates the effective rank of a matrix.

        Args:
            matrix_A (torch.Tensor): Matrix of shape m x n. 
            center_matrix_A (bool): Center the matrix 

        Returns:
            torch.Tensor: Effective rank of matrix_A
        """
        assert matrix_A.ndim == 2
        _, s, _ = torch.pca_lowrank(matrix_A, center=center_matrix_A, niter=1, q=min(matrix_A.shape[0], matrix_A.shape[1]))
        # s = torch.square(s) / (s.shape[0] - 1)
        # normalizes input s -> scale independent!
        return torch.exp(torch.distributions.Categorical(s).entropy())

In [ ]:
er = erank_svd(dir_matrix)
er

In [ ]:
# er.backward()

## How does the gradient of the erank look like?

Context: During the experiments with erank regularized Reptile I encountered issues with Inf loss values due to (very likely) Inf gradients. 
These Inf gradients occured when we evaulated the erank at the origin. In the first inner iteration we subtract the base model from the current model parameters. This results to a zero vector.
Hypothesis: 
The erank is not continuous differentiable! Especially not at the origin. Check this.

In [ ]:
norm_dir_matrix = dir_matrix / torch.linalg.norm(dir_matrix, ord=2, dim=1, keepdim=True)

In [ ]:
# torch.linalg.norm(dir_matrix, ord=2, dim=1, keepdim=True), torch.linalg.norm(norm_dir_matrix, ord=2, dim=1, keepdim=True)

In [ ]:
erank_svd(dir_matrix), erank_svd(norm_dir_matrix)

## How sensitive is the erank and its gradient to changes in the update vector? Specifically, the norm:

Here the erank and the singular values of the matrix = cat([pretrain_diffs, [norm_vec, 0,0, ...]]) is calculated.



In [ ]:
norm_update_vec = 0.0

delta = torch.zeros(size=(1,dir_matrix.shape[1])).to(dir_matrix)
delta[0] = norm_update_vec
delta.requires_grad_(True)
matrix1 = torch.cat([delta, dir_matrix], dim=0)
_, s, _ = torch.svd_lowrank(matrix1)
erank_val = erank_svd(matrix1)
erank_val.backward()
erank_grad_norm = torch.linalg.norm(delta.grad)
s, erank_val, erank_grad_norm

In [ ]:
norm_update_vec = 5.0

delta = torch.zeros(size=(1,dir_matrix.shape[1])).to(dir_matrix)
delta[0] = norm_update_vec
delta.requires_grad_(True)
matrix1 = torch.cat([delta, dir_matrix], dim=0)
_, s, _ = torch.svd_lowrank(matrix1)
erank_val = erank_svd(matrix1)
erank_val.backward()
erank_grad_norm = torch.linalg.norm(delta.grad)
s, erank_val, erank_grad_norm

The erank is sensitive to length of update vectors.

## erank plotting functions

In [ ]:
# This generates values for erank plots.
# It analyzes the erank of a matrix if the norm of one vector is increased.

# I use the directions matrix (i.e. concatenated models) and concatenate one vector which has only one entry. 
# This vector spans a one-dimensional space (a line). The value of this entry is increased.

def erank_sv_norm_update_vector_sweep(dir_matrix, update_vec, erank_fn=erank_svd, normalize_dir_matrix: bool = False, xlim_logscale=[-10,4], n_points=30):
    assert update_vec.shape == (1, dir_matrix.shape[1])
    dir_matrix.requires_grad_(False)
    erank_vals = []
    erank_grad_norm_vals = []
    singular_vals = []
    vec_norms = torch.logspace(xlim_logscale[0], xlim_logscale[1], n_points)
    for n in vec_norms:
        delta = update_vec.to(dir_matrix)
        delta = n*delta
        delta.requires_grad_(True)
        matrix1 = torch.cat([delta, dir_matrix], dim=0)
        if normalize_dir_matrix:
            # normalize matrix
            matrix1 = matrix1 / torch.linalg.norm(matrix1, ord=2, dim=1, keepdim=True)
        # calculate erank
        erank_val = erank_fn(matrix1)
        erank_vals.append(erank_val.item())
        # calculate erank grad norm
        erank_val.backward()
        erank_grad_norm_vals.append(torch.linalg.norm(delta.grad).item())
        # calculate singular values
        _, s, _ = torch.svd_lowrank(matrix1)
        singular_vals.append(s.detach().cpu().numpy())
        ret_dict = {
            'vec_norms': vec_norms,
            'erank_vals': erank_vals,
            'singular_vals': singular_vals,
            'erank_grad_norm_vals': erank_grad_norm_vals
        }
    return ret_dict

In [ ]:
def make_plot(out_subspace: Dict[str, float], in_subspace: Dict[str, float], singular_values_ylim=[20,1000], erank_ylim=[0,13]):
    fig = plt.figure(figsize=(25,14))
    fig.suptitle('First row: direction outside subspace | Second row: direction inside subspace')
    vec_norms = out_subspace['vec_norms']
    plt.subplot(2,3,1)
    plt.plot(vec_norms, out_subspace['erank_vals'], 'o-')
    plt.xscale('log')
    plt.ylabel('erank')
    plt.xlabel('norm update vector')
    plt.title('erank')
    plt.ylim(erank_ylim)
    
    plt.subplot(2,3,2)
    plt.plot(vec_norms, out_subspace['erank_grad_norm_vals'], 'o-')
    plt.xscale('log')
    plt.ylabel('grad norm erank')
    plt.xlabel('norm update vector')
    plt.title('gradient norm of concatenated vector')
    
    plt.subplot(2,3,3)
    plt.plot(vec_norms, out_subspace['singular_vals'], 'o-')
    plt.xscale('log')
    plt.yscale('log')
    plt.ylabel('singular value')
    plt.xlabel('norm update vector')
    plt.title('singular values')
    plt.ylim(singular_values_ylim)
    
    plt.subplot(2,3,4)
    plt.plot(vec_norms, in_subspace['erank_vals'], 'o-')
    plt.xscale('log')
    plt.ylabel('erank')
    plt.xlabel('norm update vector')
    plt.title('erank')
    plt.ylim(erank_ylim)
    
    plt.subplot(2,3,5)
    plt.plot(vec_norms, in_subspace['erank_grad_norm_vals'], 'o-')
    plt.xscale('log')
    plt.ylabel('grad norm erank')
    plt.xlabel('norm update vector')
    plt.title('gradient norm of concatenated vector')
    
    plt.subplot(2,3,6)
    plt.plot(vec_norms, in_subspace['singular_vals'], 'o-')
    plt.xscale('log')
    plt.yscale('log')
    plt.ylabel('singular value')
    plt.xlabel('norm update vector')
    plt.title('singular values')
    plt.ylim(singular_values_ylim)

## Unnormalized Direction Matrix M - axis parallel dir

In [ ]:
#! erank_svd
xlim_logscale=[-7,4]
normalize_dir_matrix=False
# outside subspace: axis parallel direction
update_vec1 = torch.zeros(size=(1, dir_matrix.shape[1]), requires_grad=False)
update_vec1[0,0] = 1.
out_subspace = erank_sv_norm_update_vector_sweep(dir_matrix, update_vec1, erank_fn=erank_svd, normalize_dir_matrix=normalize_dir_matrix, xlim_logscale=xlim_logscale)
# inside subspace: mean vec of dir_matrix
update_vec2 = dir_matrix.mean(dim=0, keepdim=True)
in_subspace = erank_sv_norm_update_vector_sweep(dir_matrix, update_vec2, erank_fn=erank_svd, normalize_dir_matrix=normalize_dir_matrix, xlim_logscale=xlim_logscale)
make_plot(out_subspace, in_subspace)
np.isnan(np.array(out_subspace['erank_grad_norm_vals'])).any(),np.isnan(np.array(in_subspace['erank_grad_norm_vals'])).any()

In [ ]:
#! erank_pca_lr
xlim_logscale=[-7,4]
normalize_dir_matrix=False
# outside subspace: axis parallel direction
update_vec1 = torch.zeros(size=(1, dir_matrix.shape[1]), requires_grad=False)
update_vec1[0,0] = 1.
out_subspace = erank_sv_norm_update_vector_sweep(dir_matrix, update_vec1, erank_fn=erank_pca_lr, normalize_dir_matrix=normalize_dir_matrix, xlim_logscale=xlim_logscale)
# inside subspace: mean vec of dir_matrix
update_vec2 = dir_matrix.mean(dim=0, keepdim=True)
in_subspace = erank_sv_norm_update_vector_sweep(dir_matrix, update_vec2, erank_fn=erank_pca_lr, normalize_dir_matrix=normalize_dir_matrix, xlim_logscale=xlim_logscale)
make_plot(out_subspace, in_subspace)
np.isnan(np.array(out_subspace['erank_grad_norm_vals'])).any(),np.isnan(np.array(in_subspace['erank_grad_norm_vals'])).any()

## Normalized Direction Matrix M - axis parallel dir

In [ ]:
xlim_logscale=[-7,4]
normalize_dir_matrix=True
# outside subspace: axis parallel direction
update_vec1 = torch.zeros(size=(1, dir_matrix.shape[1]), requires_grad=False)
update_vec1[0,0] = 1.
out_subspace = erank_sv_norm_update_vector_sweep(dir_matrix, update_vec1, erank_fn=erank_svd, normalize_dir_matrix=normalize_dir_matrix, xlim_logscale=xlim_logscale)
# inside subspace: mean vec of dir_matrix
update_vec2 = dir_matrix.mean(dim=0, keepdim=True)
in_subspace = erank_sv_norm_update_vector_sweep(dir_matrix, update_vec2, erank_fn=erank_svd, normalize_dir_matrix=normalize_dir_matrix, xlim_logscale=xlim_logscale)
make_plot(out_subspace, in_subspace)
np.isnan(np.array(out_subspace['erank_grad_norm_vals'])).any(),np.isnan(np.array(in_subspace['erank_grad_norm_vals'])).any()

## Unnormalized M - Random direction

In [ ]:
xlim_logscale=[-19,4]
normalize_dir_matrix=False
# outside subspace: axis parallel direction
update_vec1 = torch.normal(0, 1, size=(1, dir_matrix.shape[1]), requires_grad=False)
out_subspace = erank_sv_norm_update_vector_sweep(dir_matrix, update_vec1, erank_fn=erank_svd, normalize_dir_matrix=normalize_dir_matrix, xlim_logscale=xlim_logscale)
# inside subspace: mean vec of dir_matrix
update_vec2 = dir_matrix.mean(dim=0, keepdim=True)
in_subspace = erank_sv_norm_update_vector_sweep(dir_matrix, update_vec2, erank_fn=erank_svd, normalize_dir_matrix=normalize_dir_matrix, xlim_logscale=xlim_logscale)
make_plot(out_subspace, in_subspace)
np.isnan(np.array(out_subspace['erank_grad_norm_vals'])).any(),np.isnan(np.array(in_subspace['erank_grad_norm_vals'])).any()

In [ ]:
xlim_logscale=[-9,4]
normalize_dir_matrix=False
# outside subspace: axis parallel direction
update_vec1 = torch.normal(0, 1, size=(1, dir_matrix.shape[1]), requires_grad=False)
out_subspace = erank_sv_norm_update_vector_sweep(dir_matrix, update_vec1, erank_fn=erank_pca_lr, normalize_dir_matrix=normalize_dir_matrix, xlim_logscale=xlim_logscale)
# inside subspace: mean vec of dir_matrix
update_vec2 = dir_matrix.mean(dim=0, keepdim=True)
in_subspace = erank_sv_norm_update_vector_sweep(dir_matrix, update_vec2, erank_fn=erank_pca_lr, normalize_dir_matrix=normalize_dir_matrix, xlim_logscale=xlim_logscale)
make_plot(out_subspace, in_subspace)
np.isnan(np.array(out_subspace['erank_grad_norm_vals'])).any(),np.isnan(np.array(in_subspace['erank_grad_norm_vals'])).any()

In [ ]:
# can we compute erank with svd on zero vector?

norm_update_vec = 0.0

delta = torch.zeros(size=(1,dir_matrix.shape[1])).to(dir_matrix)
# delta[0] = norm_update_vec
delta.requires_grad_(True)
matrix1 = torch.cat([delta, dir_matrix], dim=0)
_, s, _ = torch.linalg.svd(matrix1)
erank_val = erank_svd(matrix1)
erank_val.backward()
erank_grad_norm = torch.linalg.norm(delta.grad)
s, erank_val, erank_grad_norm

### Conclusion
In this notebook we analyzed the norm of the gradients of the erank with respect to the (concatenated) update vector. The motivation were the non-smooth behavior of the erank around the origin. We want to find out how we can perturb the parameters such that we stay in the smooth region of the erank.

The erank function looks differently, depending on the direction of the input parameter (See especially the plots for the unnormalized directions matrix M):
- If we go in directions outside the subspace, the erank function has a peak at around the lengths of the other vectors.
- If we go in directions inside the subspace, this peak does not occur.

This has some implications for the noise we want to add:
We assume that a random Gauss vector is not inside the subspace. We add a random Normal distributed vector scaled by epsilon = 1e-6 to avoid numeric instabilities.

The observations are made with a random normal distributed direction matrix (a random subspace) and a subspace spanned by pretrained models on f_mnist.

### Compare: Function to compute singular values: torch.linalg.svd vs. torch.pca_lowrank
-> use torch.linalg.svd!!!